In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph


# Step-by-Step 개념 이해하기!

## STEP 1. 상태(State) 정의

In [2]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    # 메시지 정의(list type이며 add_messages 함수를 사용하여 메시지를 추가
    messages: Annotated[list, add_messages]

## STEP 2. 노드(Node) 정의

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 챗봇 함수 정의
def chatbot(state: State):
    # 메시지 호출 및 반환
    return {"messages": [llm.invoke(state["messages"])]}

## STEP 3. 그래프(Graph) 정의, 노드 추가

In [5]:
# 그래프 생성
graph_builder = StateGraph(State)

# 노드 이름, 함수 혹은 callable 객체를 인자로 받아 노드를 추가
graph_builder.add_node("chatbot", chatbot)

# STEP 4. 그래프 엣지(Edge) 추가

In [6]:
# 시작 노드에서 챗봇 노드로의 엣지 추가
graph_builder.add_edge(START, "chatbot")

In [7]:
# 그래프에 엣지 추가
graph_builder.add_edge("chatbot", END)

## STEP 5. 그래프 컴파일(compile)

In [8]:
# 그래프 컴파일
graph = graph_builder.compile()

## STEP 6. 그래프 시각화

In [18]:
# %%capture --no-stderr
from langchain_teddynote.graphs import visualize_graph

# 그래프 시각화
visualize_graph(graph)

[ERROR] Visualize Graph Error: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)


## STEP 7. 그래프 실행

In [20]:
question = "서울의 유명한 맛집 TOP 10 추천해줘"

# 그래프 이벤트 스트리밍
for event in graph.stream({"messages": [("user", question)]}):
    # 이벤트 값 출력
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: 서울에는 다양한 맛집이 많아서 선택하기가 쉽지 않지만, 다음은 서울에서 유명한 맛집 TOP 10을 추천해 드립니다.

1. **광장시장** - 전통 시장으로, 빈대떡, 마약김밥, 떡볶이 등 다양한 길거리 음식을 즐길 수 있습니다.

2. **이태원 부대찌개** - 부대찌개로 유명한 이태원의 맛집으로, 푸짐한 양과 깊은 맛이 특징입니다.

3. **명동교자** - 칼국수와 만두가 유명한 곳으로, 항상 많은 사람들이 줄 서서 기다리는 인기 맛집입니다.

4. **삼청동 수제비** - 수제비와 전통 한식을 즐길 수 있는 곳으로, 아늑한 분위기에서 식사를 할 수 있습니다.

5. **한남동 고기리 막창** - 막창과 고기를 전문으로 하는 맛집으로, 신선한 재료와 맛있는 양념이 특징입니다.

6. **을지로 골뱅이** - 골뱅이 무침과 소주를 함께 즐길 수 있는 곳으로, 분위기가 독특하고 맛이 뛰어납니다.

7. **홍대 돈부리** - 일본식 덮밥 전문점으로, 다양한 종류의 돈부리를 맛볼 수 있습니다.

8. **신사동 가로수길** - 다양한 카페와 레스토랑이 모여 있는 곳으로, 특히 브런치 맛집이 많습니다.

9. **종로 통인시장** - 전통 시장으로, 다양한 한식과 간식을 즐길 수 있는 곳입니다. 특히 찐빵과 떡이 유명합니다.

10. **압구정 로데오거리** - 고급 레스토랑과 카페가 많은 지역으로, 다양한 세계 요리를 맛볼 수 있습니다.

이 외에도 서울에는 많은 맛집이 있으니, 방문하실 때마다 새로운 곳을 탐방해 보세요!
